# Прикрепите файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import folium
from tqdm import tqdm
import requests
from ipywidgets import FileUpload, Button, Output, VBox
from IPython.display import display, FileLink, HTML
import os
import warnings
warnings.filterwarnings('ignore')

processed_data = None
def on_upload_change(change):
    global processed_data
    if not upload.value:
        return
    
    # Получаем загруженный файл
    uploaded_file = next(iter(upload.value.values()))
    #print(uploaded_file)
    # Здесь можно добавить вашу логику обработки файла
    # Пример: просто сохраняем содержимое как есть
    processed_data = pd.read_excel(uploaded_file['content'])
    
    print(f"Файл успешно загружен!")
    
def on_process_click(btn):
    if processed_data is None:
        print("Сначала загрузите файл!")
        return
    
    # Создаем временный файл для скачивания
    output_file = "optimized_routes.html"
    #with open(output_file, 'wb') as f:
    #    f.write(processed_data)
    
    # Показываем ссылку для скачивания
    display(HTML(
            f'<a href="{output_file}" download="{output_file}">'
            f'Нажмите для скачивания</a>'
        ))
    
upload = FileUpload(description="Загрузить файл", multiple=False)
process_btn = Button(description="Скачать файл")
download_output = Output()

upload.observe(on_upload_change, names='value')
#process_btn.on_click(on_process_click)
display(VBox([upload]))#, process_btn, download_output]))

C:\Users\Marina\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Marina\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\Marina\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


load C:\Users\Marina\anaconda3\lib\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\Marina\anaconda3\lib\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\Marina\anaconda3\lib\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\Marina\anaconda3\lib\site-packages\ortools\.libs\re2.dll...
load C:\Users\Marina\anaconda3\lib\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\Marina\anaconda3\lib\site-packages\ortools\.libs\highs.dll...
load C:\Users\Marina\anaconda3\lib\site-packages\ortools\.libs\ortools.dll...


Файл успешно загружен!


In [ ]:
def get_road_distance(cli, fac):
    lat1 = cli[0]
    lon1 = cli[1]
    lat2 = fac[0]
    lon2 = fac[1]
    alpha = 0.5
    url = f"http://router.project-osrm.org/route/v1/car/{lon1},{lat1};{lon2},{lat2}?overview=false"
    r = requests.get(url).json()
    return r['routes'][0]['duration'] / 60

def euclidean(cli, fac):
    return np.sqrt((cli[0] - fac[0]) **2 + (cli[1] - fac[1])**2)

order = pd.read_excel('sample_order.xlsx')
order = processed_data.copy()
order = pd.DataFrame(order)
order_leftover = order[order['Долгота'] >= 60]
order = order[order['Долгота'] < 60]

factories = {
    'Волхов': {'Широта': 59.9177, 'Долгота': 32.3122},
    'Пенза': {'Широта': 53.1958, 'Долгота': 45.0000}
}


client_coords = (order[['Широта', 'Долгота']].values)
fc = []
for f in factories.values():
    fc.append([f['Широта'], f['Долгота']])
factory_coords = np.array(fc)

distances = cdist(client_coords, factory_coords, metric=euclidean)#get_distance_matrix)
order['Объем, тн'] = order['Объем, тн'] * 1000
order['Начало'] = (order['Начало окна доставки'] - pd.to_datetime('today')).dt.components['days'] * 24 * 60 + (order['Начало окна доставки'] - pd.to_datetime('today')).dt.components['hours'] * 60 + 24 * 60
order['Конец'] = (order['Конец окна доставки'] - pd.to_datetime('today')).dt.components['days'] * 24 * 60 + (order['Конец окна доставки'] - pd.to_datetime('today')).dt.components['hours'] * 60 + 24 * 60
order = order.drop(columns=['Дата доставки', 'Начало окна доставки', 'Конец окна доставки'])
order['завод'] = [list(factories.keys())[i] for i in np.argmin(distances, axis=1)]

order_volkhov = order[order['завод'] == 'Волхов'].copy()
order_volkhov.loc[-1] = ['Ромашка', 'Ромашка Волхов (завод)', 'Волхов (завод)', 59.9177, 32.3122, 0, 0, 999999, 'Волхов']

order_volkhov.index = order_volkhov.index + 1  # shifting index
order_volkhov = order_volkhov.sort_index()
order_volkhov = order_volkhov.reset_index(drop=True)

order_penza = order[order['завод'] == 'Пенза'].copy()
order_penza.loc[-1] = ['Ромашка', 'Ромашка Пенза (завод)', 'Пенза (завод)', 53.1958, 45.0000, 0, 0, 999999, 'Пенза']

order_penza.index = order_penza.index + 1  # shifting index
order_penza = order_penza.sort_index()
order_penza = order_penza.reset_index(drop=True)

order_all = order.copy()
order_all.loc[-1] = ['Ромашка', 'Ромашка', 'Пенза (завод)', 53.1958, 45.0000, 0, 0, 999999, 'Пенза']  # adding a row
order_all.index = order_all.index + 1  # shifting index
order_all = order_all.sort_index()
order_all = order_all.reset_index(drop=True)

def get_distance_matrix(order):
    client_coords = (order[['Широта', 'Долгота']].values)
    M = np.zeros((len(client_coords), len(client_coords)))
    for i in range(len(client_coords)):
        for j in range(len(client_coords)):
            if i == j:
                M[i][j] = 0
            else:
                #M[i][j] = int(euclidean(client_coords[i], client_coords[j]) / 70) + 1 #get_road_distance(client_coords[i], client_coords[j])
                M[i][j] = int(get_road_distance(client_coords[i], client_coords[j])) + 30
    return M.tolist()

def get_time_windows(order):
    begin_time = (order['Начало'].values)
    end_time = (order['Конец'].values)
    M = []
    for i in range(len(begin_time)):
        if int(end_time[i]) == 99999:
            M.append((0, 999999))
        else:
            M.append((int(begin_time[i]), int(end_time[i])))
    return M

def index_to_city(index, order):
    return order.loc[index]['КА']

def index_to_coord(index, order):
    return [order.loc[index]['Широта'], order.loc[index]['Долгота']]

def print_solution(data, manager, routing, solution, order):
    total_distance = 0
    total_load = 0
    ans = []
    time_dimension = routing.GetDimensionOrDie('Time')
    vehicle_count = 0
    avg_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        index1 = index
        route_distance = 0
        route_load = 0
        route_load1 = 0
        lats = []
        lons = []
        while not routing.IsEnd(index1):
            node_index1 = manager.IndexToNode(index1)
            route_load1 += data["demands"][node_index1]
            index1 = solution.Value(routing.NextVar(index1))
        load =  route_load1
        if load != 0:
            vehicle_count += 1
            print('---------------------------------------------------------------------------------------------')
            plan_output = f"Путь для машины {index_to_city(0, order)[:-8]} {vehicle_count}:\n\n"
            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                route_load += data["demands"][node_index]
                if node_index == 0:
                    x = (22 - len(index_to_city(manager.IndexToNode(index), order)[:-8])) * ' '
                    plan_output += f" {index_to_city(manager.IndexToNode(index), order)[:-8]}{x}| Нагрузка {load}кг = {round(load / data['vehicle_capacities'][vehicle_id] * 100)}% ->\n"
                else:
                    x = (20 - len(index_to_city(node_index, order))) * ' '
                    plan_output += f"   {index_to_city(node_index, order)}{x}| Время прибытия {pd.to_datetime('today').normalize() + pd.Timedelta(minutes=int(solution.Value(time_dimension.CumulVar(index))))} | Запрос {route_load - previous_load}кг | Нагрузка {load}кг ->\n"
                    load -= (route_load - previous_load)
                previous_load = route_load
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                lats.append(index_to_coord(node_index, order)[0])
                lons.append(index_to_coord(node_index, order)[1])
                distance = routing.GetArcCostForVehicle(
                    previous_index, index, vehicle_id
                )
                route_distance += distance
            x = (22 - len(index_to_city(manager.IndexToNode(index), order)[:-8])) * ' '
            plan_output += f" {index_to_city(manager.IndexToNode(index), order)[:-8]}{x}| Время прибытия {pd.to_datetime('today').normalize() + pd.Timedelta(minutes=int(solution.Value(time_dimension.CumulVar(index))))} | Нагрузка {load}кг\n"
            plan_output += f"Время пути: {route_distance // 60}ч {route_distance % 60} мин\n"
            plan_output += f"Нагрузка пути: {route_load}кг = {round(route_load / data['vehicle_capacities'][vehicle_id] * 100)}%\n"
            lats.append(index_to_coord(manager.IndexToNode(index), order)[0])
            lons.append(index_to_coord(manager.IndexToNode(index), order)[1])
            ans.append([lats, lons])
            print(plan_output)
            total_distance += route_distance
            total_load += route_load
            avg_load += round(route_load / data['vehicle_capacities'][vehicle_id] * 100)
    print('=============================================================================================')
    print(f"Суммарное время пути: {route_distance // 60}ч {route_distance % 60} мин")
    print(f"Суммарная нагрузка: {total_load}кг")
    if vehicle_count != 0:
        print(f"Средняя нагрузка: {round(avg_load / vehicle_count)}%")
    print('=============================================================================================')
    print()
    return ans

def draw_solution_folium(data1, manager1, routing1, solution1, order1, data2, manager2, routing2, solution2, order2):
    # Координаты центрирования карты (среднее между заводами)
    factories = {
        'Волхов': {'Широта': 59.9177, 'Долгота': 32.3122},
        'Пенза': {'Широта': 53.1958, 'Долгота': 45.0000}
    }
    center_lat = (factories['Волхов']['Широта'] + factories['Пенза']['Широта']) / 2
    center_lon = (factories['Волхов']['Долгота'] + factories['Пенза']['Долгота']) / 2
    
    # Создаем карту
    m = folium.Map(location=[center_lat, center_lon], zoom_start=6)
    
    # Цвета для заводов
    colors = {'Волхов': 'red', 'Пенза': 'green'}
    
    # Добавляем клиентов
    for factory, color in colors.items():
        factory_clients = order_all[order_all['завод'] == factory]
        for idx, row in factory_clients.iterrows():
            folium.CircleMarker(
                location=[row['Широта'], row['Долгота']],
                radius=7,  # Масштабируем размер по объему
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.4,
                popup=f"Клиент: {row.name}<br>Объем: {row['Объем, тн']} тн<br>Завод: {factory}",
                tooltip=f"{row.name} ({row['Объем, тн']} тн)"
            ).add_to(m)
    
    # Добавляем заводы (с иконками фабрик)
    for name, loc in factories.items():
        folium.Marker(
            location=[loc['Широта'], loc['Долгота']],
            icon=folium.Icon(icon='industry', prefix='fa', color='black'),
            popup=f"Завод: {name}",
            tooltip=f"Завод {name}"
        ).add_to(m)
    
    # Получаем маршруты из решений
    ll1 = print_solution(data1, manager1, routing1, solution1, order1)
    ll2 = print_solution(data2, manager2, routing2, solution2, order2)
    # Добавляем маршруты на карту
    route_colors = ['blue', 'purple', 'red', 'orange', 'yellow', 'green', 'black', 'purple', 'brown'] * max(len(ll1), len(ll2))  # Цвета для разных маршрутов
    
    for i, segment in enumerate(ll1):
        folium.PolyLine(
            locations=[[lat, lon] for lat, lon in zip(segment[0], segment[1])],
            color=route_colors[i],
            weight=3,
            opacity=0.7,
            popup=f"Маршрут {i+1}",
            tooltip=f"Путь {i+1}"
        ).add_to(m)
    

    for i, segment in enumerate(ll2):
        folium.PolyLine(
            locations=[[lat, lon] for lat, lon in zip(segment[0], segment[1])],
            color=route_colors[i],
            weight=3,
            opacity=0.7,
            popup=f"Маршрут {i+1}",
            tooltip=f"Путь {i+1}"
        ).add_to(m)
    
    # Добавляем слой для управления отображением
    folium.LayerControl().add_to(m)
    
    # Сохраняем карту
    m.save('optimized_routes.html')
    #print("Интерактивная карта сохранена как 'optimized_routes.html'")
    return m

def create_data_model(order):
    """Stores the data for the problem."""
    data = {}
    data["time_matrix"] = get_distance_matrix(order)
    data["demands"] = order['Объем, тн']
    
    if order['завод'][0] == 'Пенза':
        data["vehicle_capacities"] = [10000] * 10
        data["num_vehicles"] = 10
    if order['завод'][0] == 'Волхов':
        data["vehicle_capacities"] = [10000] * 10
        data["num_vehicles"] = 10
        
    data["depot"] = 0
    data["time_windows"] = get_time_windows(order)
    return data

def main():
    # Create and register a transit callback.
    def distance_callback1(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager1.IndexToNode(from_index)
        to_node = manager1.IndexToNode(to_index)
        return data1["time_matrix"][from_node][to_node]
    
    # Add Capacity constraint.
    def demand_callback1(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager1.IndexToNode(from_index)
        return data1["demands"][from_node]
    
    def distance_callback2(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager2.IndexToNode(from_index)
        to_node = manager2.IndexToNode(to_index)
        return data2["time_matrix"][from_node][to_node]
    
    # Add Capacity constraint.
    def demand_callback2(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager2.IndexToNode(from_index)
        return data2["demands"][from_node]
    
    
    order1 = order_penza.copy()
    data1 = create_data_model(order1)

    manager1 = pywrapcp.RoutingIndexManager(
        len(data1["time_matrix"]), data1["num_vehicles"], data1["depot"]
    )

    # Create Routing Model.
    routing1 = pywrapcp.RoutingModel(manager1)
    transit_callback_index1 = routing1.RegisterTransitCallback(distance_callback1)
    routing1.SetArcCostEvaluatorOfAllVehicles(transit_callback_index1)

    time1 = 'Time'
    routing1.AddDimension(
        transit_callback_index1,
        30,  # время ожидания (максимальное время ожидания в точке)
        999999,  # максимальное время работы транспортного средства
        False,  # не начинать с нуля
        time1,
    )
    time_dimension1 = routing1.GetDimensionOrDie(time1)
    
    # Добавляем ограничения временных окон для каждой точки
    for location_idx, time_window in enumerate(data1['time_windows']):
        if location_idx == data1['depot']:
            continue
        index = manager1.NodeToIndex(location_idx)
        time_dimension1.CumulVar(index).SetRange(time_window[0], time_window[1])
    
    # Добавляем ограничения временных окон для депо
    depot_idx = data1['depot']
    solver1 = routing1.solver()
    for vehicle_id in range(data1['num_vehicles']):
        index = routing1.Start(vehicle_id)
        time_dimension1.CumulVar(index).SetRange(
            data1['time_windows'][depot_idx][0],
            data1['time_windows'][depot_idx][1],
        )
        start_index = routing1.Start(vehicle_id)
        end_index = routing1.End(vehicle_id)
        start_cumul = time_dimension1.CumulVar(start_index)
        end_cumul = time_dimension1.CumulVar(end_index)
        # Продолжительность = конечное время - начальное время <= 9
        solver1.Add(end_cumul - start_cumul <= 10 * 60)

    
    demand_callback_index1 = routing1.RegisterUnaryTransitCallback(demand_callback1)
    routing1.AddDimensionWithVehicleCapacity(
        demand_callback_index1,
        0,  # null capacity slack
        data1["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters1 = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters1.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters1.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters1.time_limit.FromSeconds(1)

    # Solve the problem.
    solution1 = routing1.SolveWithParameters(search_parameters1)
    
    
    
    order2 = order_volkhov.copy()
    data2 = create_data_model(order2)

    manager2 = pywrapcp.RoutingIndexManager(
        len(data2["time_matrix"]), data2["num_vehicles"], data2["depot"]
    )

    # Create Routing Model.
    routing2 = pywrapcp.RoutingModel(manager2)
    transit_callback_index2 = routing2.RegisterTransitCallback(distance_callback2)
    routing2.SetArcCostEvaluatorOfAllVehicles(transit_callback_index2)

    time2 = 'Time'
    routing2.AddDimension(
        transit_callback_index2,
        30,  # время ожидания (максимальное время ожидания в точке)
        999999,  # максимальное время работы транспортного средства
        False,  # не начинать с нуля
        time2,
    )
    time_dimension2 = routing2.GetDimensionOrDie(time2)
    
    # Добавляем ограничения временных окон для каждой точки
    for location_idx, time_window in enumerate(data2['time_windows']):
        if location_idx == data2['depot']:
            continue
        index = manager2.NodeToIndex(location_idx)
        time_dimension2.CumulVar(index).SetRange(time_window[0], time_window[1])
    
    # Добавляем ограничения временных окон для депо
    depot_idx = data2['depot']
    solver2 = routing2.solver()
    for vehicle_id in range(data2['num_vehicles']):
        index = routing2.Start(vehicle_id)
        time_dimension2.CumulVar(index).SetRange(
            data2['time_windows'][depot_idx][0],
            data2['time_windows'][depot_idx][1],
        )
        start_index = routing2.Start(vehicle_id)
        end_index = routing2.End(vehicle_id)
        start_cumul = time_dimension2.CumulVar(start_index)
        end_cumul = time_dimension2.CumulVar(end_index)
        # Продолжительность = конечное время - начальное время <= 9
        solver2.Add(end_cumul - start_cumul <= 10 * 60)
    
    demand_callback_index2 = routing2.RegisterUnaryTransitCallback(demand_callback2)
    routing2.AddDimensionWithVehicleCapacity(
        demand_callback_index2,
        0,  # null capacity slack
        data2["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )
    
    # Setting first solution heuristic.
    search_parameters2 = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters2.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters2.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters2.time_limit.FromSeconds(1)
    
    # Solve the problem.
    solution2 = routing2.SolveWithParameters(search_parameters2)

    # Print solution on console.
    if solution1 and solution2:
        #draw_solution(data1, manager1, routing1, solution1, order1, data2, manager2, routing2, solution2, order2)
        draw_solution_folium(data1, manager1, routing1, solution1, order1, data2, manager2, routing2, solution2, order2)
    else:
        if solution1:
            print('No solution Volkhov')
        else:
            if solution2:
                print('No solution Penza')
            else:
                print('No solution Penza & Volkhov')
        
    if len(order_leftover) != 0:
        print()
        print('Отправляются Ж/Д транспортом')
        codes = order_leftover['Код клиента'].values
        regions = order_leftover['регион'].values
        for i in range(len(codes)):
            print(f"{codes[i]} ({regions[i]})")
        
    


if __name__ == "__main__":
    main()
    
process_btn.on_click(on_process_click)
display(VBox([process_btn, download_output]))